In [ ]:
#!wget -P /content/drive/"My Drive"/"ML Offline" "https://www.dropbox.com/s/1jdct708qk8p6za/Data.zip?dl=0"

--2020-10-09 11:20:48--  https://www.dropbox.com/s/1jdct708qk8p6za/Data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1jdct708qk8p6za/Data.zip [following]
--2020-10-09 11:20:48--  https://www.dropbox.com/s/raw/1jdct708qk8p6za/Data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4209430171ee7eb4c607001b73.dl.dropboxusercontent.com/cd/0/inline/BA40yfMZoBkDUFYmw8Su5xVhtKWN3KBMPke_Bm8wdGvLoXVh6GwjiowkeWXfTpM0YvN8kc-1MOy8MrkxqijmO_gtQEZdJiERZW1-_h_uqarQgwugTUI6c7KXaC9GSfT2mTM/file# [following]
--2020-10-09 11:20:49--  https://uc4209430171ee7eb4c607001b73.dl.dropboxusercontent.com/cd/0/inline/BA40yfMZoBkDUFYmw8Su5xVhtKWN3KBMPke_Bm8wdGvLoXVh6GwjiowkeWXfTpM0YvN8kc-1MOy8MrkxqijmO_gtQEZdJiERZW1-_h_uqarQgwugTUI6c7KXa

In [ ]:
import shutil
shutil.copy2("/content/drive/My Drive/ML Offline/Data.zip?dl=0", "/content")

'/content/Data.zip?dl=0'

In [ ]:
!unzip /content/drive/"My Drive"/"ML Offline"/Data.zip?dl=0

Archive:  /content/drive/My Drive/ML Offline/Data.zip?dl=0
   creating: Data/Test/
  inflating: Data/Test/3d_Printer.xml  
  inflating: Data/Test/Anime.xml     
  inflating: Data/Test/Arduino.xml   
  inflating: Data/Test/Astronomy.xml  
  inflating: Data/Test/Biology.xml   
  inflating: Data/Test/Chess.xml     
  inflating: Data/Test/Coffee.xml    
  inflating: Data/Test/Cooking.xml   
  inflating: Data/Test/Law.xml       
  inflating: Data/Test/Space.xml     
  inflating: Data/Test/Windows_Phone.xml  
  inflating: Data/Test/Wood_Working.xml  
  inflating: Data/topics.txt         
   creating: Data/Training/
  inflating: Data/Training/3d_Printer.xml  
  inflating: Data/Training/Anime.xml  
  inflating: Data/Training/Arduino.xml  
  inflating: Data/Training/Astronomy.xml  
  inflating: Data/Training/Biology.xml  
  inflating: Data/Training/Chess.xml  
  inflating: Data/Training/Coffee.xml  
  inflating: Data/Training/Cooking.xml  
  inflating: Data/Training/Law.xml   
  inflating: Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
from sys import stdout
import time
import os
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def preprocess(text):
  #Lowercase the text
  text = text.lower()

  #Number Removal
  text = re.sub(r'[-+]?\d+', '', text)

  #Remove punctuations
  text=text.translate((str.maketrans('','',string.punctuation)))

  #Tokenize
  text = word_tokenize(text)

  #Remove stopwords
  stop_words = set(stopwords.words('english'))
  text = [word for word in text if not word in stop_words]

  #Lemmatize tokens
  lemmatizer=WordNetLemmatizer()
  text = [lemmatizer.lemmatize(word) for word in text]

  #Stemming tokens
  stemmer= PorterStemmer()
  text = [stemmer.stem(word) for word in text]
  return text

In [ ]:
data_path = "/content/Data/Training"
data_files_list = os.listdir(data_path)
data_files_list.remove('3d_Printer.xml')
#data_files_list = ['Coffee.xml', 'Arduino.xml', 'Anime.xml']
training_data_dict_list = []
valid_data_dict_list = []
test_data_dict_list = []
for data_file in tqdm(data_files_list):
  data_file_path = data_path + "/" + data_file
  topic = data_file.split(".")[0]
  #print(topic)
  with open(data_file_path, "r") as f:
    line_count = 0
    for line in f:
      line = line.lstrip()
      if line.startswith("<row"):
        #line_count += 1
        data_dict = {}
        soup = BeautifulSoup(line, "xml")
        body_temp = soup.find("row")["Body"]
        soup2 = BeautifulSoup(body_temp, "html")
        body = soup2.get_text().replace("\n", " ")
        if body.strip() == "":
          continue
        words_list = preprocess(body)
        if len(words_list) == 0:
          continue
        line_count += 1
        data_dict['doc'] = words_list
        data_dict['topic'] = topic
        if line_count <= 500:
          training_data_dict_list.append(data_dict)
        elif line_count > 500 and line_count <=700:
          valid_data_dict_list.append(data_dict)
        elif line_count > 700 and line_count <= 1200:
          test_data_dict_list.append(data_dict)   
      if line_count >= 1200: 
        break

100%|██████████| 11/11 [00:46<00:00,  4.23s/it]


In [ ]:
print(len(training_data_dict_list))
print(len(test_data_dict_list))
print(len(valid_data_dict_list))

5500
5500
2200


In [ ]:
with open("train.json", "w") as f:
  json.dump(training_data_dict_list, f)

with open("test.json", "w") as f:
  json.dump(test_data_dict_list, f)

with open ("valid.json", "w") as f:
  json.dump(valid_data_dict_list, f)

In [ ]:
shutil.copy2("/content/train.json", "/content/drive/My Drive/ML Offline")
shutil.copy2("/content/valid.json", "/content/drive/My Drive/ML Offline")
shutil.copy2("/content/test.json", "/content/drive/My Drive/ML Offline")

'/content/drive/My Drive/ML Offline/test.json'

In [ ]:
with open("train.json", "r") as f:
  train_data = json.load(f)

In [ ]:
with open("valid.json", "r") as f:
  valid_data = json.load(f)

In [ ]:
with open("test.json", "r") as f:
  test_data = json.load(f)

In [ ]:
all_train_words = []
for d in train_data:
  for word in d['doc']:
    all_train_words.append(word)

In [ ]:
vocabulary = np.unique(all_train_words)

In [ ]:
print(len(vocabulary))

20620


In [ ]:
binary_train_data = []
for d in train_data:
  binary_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    if v in words_in_data:
      binary_rep.append(1)
    else:
      binary_rep.append(0)
  data_dict['bin_doc'] = binary_rep
  data_dict['topic'] = d['topic']
  binary_train_data.append(data_dict)

In [ ]:
#with open("binary_rep_train.json", "w") as f:
#  json.dump(binary_train_data, f)

In [ ]:
#shutil.copy2("/content/binary_rep_train.json", "/content/drive/My Drive/ML Offline")

'/content/drive/My Drive/ML Offline/binary_rep_train.json'

In [ ]:
binary_valid_data = []
for d in tqdm(valid_data):
  binary_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    if v in words_in_data:
      binary_rep.append(1)
    else:
      binary_rep.append(0)
  data_dict['bin_doc'] = binary_rep
  data_dict['topic'] = d['topic']
  binary_valid_data.append(data_dict)


100%|██████████| 2200/2200 [01:37<00:00, 22.55it/s]


In [ ]:
#with open("binary_rep_valid.json", "w") as f:
#  json.dump(binary_valid_data, f)

In [ ]:
#shutil.copy2("/content/binary_rep_valid.json", "/content/drive/My Drive/ML Offline")

'/content/drive/My Drive/ML Offline/binary_rep_valid.json'

In [ ]:
binary_test_data = []
for d in tqdm(test_data):
  binary_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    if v in words_in_data:
      binary_rep.append(1)
    else:
      binary_rep.append(0)
  data_dict['bin_doc'] = binary_rep
  data_dict['topic'] = d['topic']
  binary_test_data.append(data_dict)


100%|██████████| 5500/5500 [03:59<00:00, 22.93it/s]


In [ ]:
X_train_binary = []
Y_train_binary = []
for d in binary_train_data:
  X_train_binary.append(d['bin_doc'])
  Y_train_binary.append(d['topic'])
X_train_binary = np.array(X_train_binary)
Y_train_binary = np.array(Y_train_binary)

In [ ]:
del(binary_train_data)

In [ ]:
X_valid_binary = []
Y_valid_binary = []
for d in binary_valid_data:
  X_valid_binary.append(d['bin_doc'])
  Y_valid_binary.append(d['topic'])
X_valid_binary = np.array(X_valid_binary)
Y_valid_binary = np.array(Y_valid_binary)

In [ ]:
del(binary_valid_data)

In [ ]:
X_test_binary = []
Y_test_binary = []
for d in binary_test_data:
  X_test_binary.append(d['bin_doc'])
  Y_test_binary.append(d['topic'])
X_test_binary = np.array(X_test_binary)
Y_test_binary = np.array(Y_test_binary)

In [ ]:
del(binary_test_data)

In [ ]:
def hamming_distance(a, b):
  return (np.sum(np.absolute(a - b)))
def euclidean_distance(instance1, instance2):
  a = instance1
  b = instance2
  return np.sqrt(np.sum(np.square(a - b)))
def cosine_similarity(a, b):
  return (np.sum(np.multiply(a, b))) / (np.sqrt((np.sum(np.square(a)))) * np.sqrt((np.sum(np.square(b)))))

In [ ]:
bow_train_data = []
for d in tqdm(train_data):
  bow_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    bow_rep.append(words_in_data.count(v))
  data_dict['bow_doc'] = bow_rep
  data_dict['topic'] = d['topic']
  bow_train_data.append(data_dict)
    

In [ ]:
bow_valid_data = []
for d in tqdm(valid_data):
  bow_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    bow_rep.append(words_in_data.count(v))
  data_dict['bow_doc'] = bow_rep
  data_dict['topic'] = d['topic']
  bow_valid_data.append(data_dict)
    

In [ ]:
bow_test_data = []
for d in test_data:
  bow_rep = []
  data_dict = {}
  words_in_data = d['doc']
  for v in vocabulary:
    bow_rep.append(words_in_data.count(v))
  data_dict['bow_doc'] = bow_rep
  data_dict['topic'] = d['topic']
  bow_test_data.append(data_dict)
    

In [ ]:
#with open("binary_rep_test.json", "w") as f:
#  json.dump(binary_test_data, f)

In [ ]:
#shutil.copy2("/content/binary_rep_test.json", "/content/drive/My Drive/ML Offline")

'/content/drive/My Drive/ML Offline/binary_rep_test.json'

In [ ]:
idf = {}
D = len(train_data)
for v in vocabulary:
  idf[v] = 0

for v in tqdm(vocabulary):
  for d in train_data:
    if v in d['doc']:
      idf[v] += 1
  idf[v] = np.log(D/idf[v])

100%|██████████| 20620/20620 [02:08<00:00, 160.97it/s]


In [ ]:
tfidf_train_data = []
for d in tqdm(train_data):
  tfidf_rep = []
  data_dict = {}
  words_in_data = d['doc']
  word_num = len(words_in_data)
  for v in vocabulary:
    word_count = words_in_data.count(v)
    if word_count == 0:
      tfidf_rep.append(0)
    else:
      rep = (word_count/word_num) * idf[v]
      tfidf_rep.append(rep)
  data_dict['tfidf_doc'] = tfidf_rep
  data_dict['topic'] = d['topic']
  tfidf_train_data.append(data_dict)

100%|██████████| 5500/5500 [03:57<00:00, 23.14it/s]


In [ ]:
tfidf_valid_data = []
for d in valid_data:
  tfidf_rep = []
  data_dict = {}
  words_in_data = d['doc']
  word_num = len(words_in_data)
  for v in vocabulary:
    word_count = words_in_data.count(v)
    if word_count == 0:
      tfidf_rep.append(0)
    else:
      rep = (word_count/word_num) * idf[v]
      tfidf_rep.append(rep)
  data_dict['tfidf_doc'] = tfidf_rep
  data_dict['topic'] = d['topic']
  tfidf_valid_data.append(data_dict)

In [ ]:
tfidf_test_data = []
for d in tqdm(test_data):
  tfidf_rep = []
  data_dict = {}
  words_in_data = d['doc']
  word_num = len(words_in_data)
  for v in vocabulary:
    word_count = words_in_data.count(v)
    if word_count == 0:
      tfidf_rep.append(0)
    else:
      rep = (word_count/word_num) * idf[v]
      tfidf_rep.append(rep)
  data_dict['tfidf_doc'] = tfidf_rep
  data_dict['topic'] = d['topic']
  tfidf_test_data.append(data_dict)

100%|██████████| 5500/5500 [04:08<00:00, 22.09it/s]


In [ ]:
X_train_bow = []
Y_train_bow = []
for d in bow_train_data:
  X_train_bow.append(d['bow_doc'])
  Y_train_bow.append(d['topic'])
X_train_bow = np.array(X_train_bow)
Y_train_bow = np.array(Y_train_bow)

In [ ]:
del(bow_train_data)

In [ ]:
X_valid_bow = []
Y_valid_bow = []
for d in bow_valid_data:
  X_valid_bow.append(d['bow_doc'])
  Y_valid_bow.append(d['topic'])
X_valid_bow = np.array(X_valid_bow)
Y_valid_bow = np.array(Y_valid_bow)

In [ ]:
del(bow_valid_data)

In [ ]:
X_test_bow = []
Y_test_bow = []
for d in bow_test_data:
  X_test_bow.append(d['bow_doc'])
  Y_test_bow.append(d['topic'])
X_test_bow = np.array(X_test_bow)
Y_test_bow = np.array(Y_test_bow)

In [ ]:
X_train_tfidf = []
Y_train_tfidf = []
for d in tfidf_train_data:
  X_train_tfidf.append(d['tfidf_doc'])
  Y_train_tfidf.append(d['topic'])
X_train_tfidf = np.array(X_train_tfidf)
Y_train_tfidf = np.array(Y_train_tfidf)

In [ ]:
X_valid_tfidf = []
Y_valid_tfidf = []
for d in tfidf_valid_data:
  X_valid_tfidf.append(d['tfidf_doc'])
  Y_valid_tfidf.append(d['topic'])
X_valid_tfidf = np.array(X_valid_tfidf)
Y_valid_tfidf = np.array(Y_valid_tfidf)

In [ ]:
X_test_tfidf = []
Y_test_tfidf = []
for d in tfidf_test_data:
  X_test_tfidf.append(d['tfidf_doc'])
  Y_test_tfidf.append(d['topic'])
X_test_tfidf = np.array(X_test_tfidf)
Y_test_tfidf = np.array(Y_test_tfidf)

In [ ]:
def log(msg):
  with open("report.txt", "a") as f:
    f.write(msg)
    f.write("\n") 

In [ ]:
def prediction(X_train, Y_train, X_test, n_neighbors, obj):
    allTestNeighbers=[]
    allPredictedOutputs =[]
    
    #Determine Number of unique class lebels
    uniqueOutputLabels = []
    for label in Y_train:
        if label not in uniqueOutputLabels:
            uniqueOutputLabels.append(label)
    uniqueOutputCount = len(uniqueOutputLabels)
    #print("Unique labels:")
    #print(uniqueOutputLabels)
    #calculate for earch test data points
    for testInput in X_test:
        allDistances = []
        for trainInput, trainActualOutput in zip(X_train, Y_train):
            if obj == "binary":
                distance = hamming_distance(testInput, trainInput)
                allDistances.append((trainInput, trainActualOutput, distance))
            elif obj == "bow":
                distance = euclidean_distance(testInput, trainInput)
                allDistances.append((trainInput, trainActualOutput, distance))
            elif obj == "tfidf":
                distance = cosine_similarity(testInput, trainInput)
                allDistances.append((trainInput, trainActualOutput, distance))
        #Sort (in ascending order) the training data points based on distances from the test point 
        if obj == "binary" or obj == "bow":    
            allDistances.sort(key=lambda x: x[2])
        else:
            allDistances.sort(key=lambda x: x[2])
            allDistances.reverse()
        
        
        #Assuming output labels are from 0 to uniqueOutputCount-1
        voteCount = {}
        for l in uniqueOutputLabels:
          voteCount[l] = 0
        
        neighbors = []
        for n in range(n_neighbors):
            neighbors.append(allDistances[n][0])
            class_label = (allDistances[n][1])
            if obj == "binary" or obj == "bow":
                voteCount[class_label] += 1 * (1/allDistances[n][2])
            else:
                voteCount[class_label] += 1 * (allDistances[n][2])
        #print("VoteCount:")
        #print(voteCount)
        #Determine the Majority Voting (Equal weight considered)
        predictedOutput = ""
        max_vote = -999999
        for k in voteCount.keys():
          if voteCount[k] > max_vote:
            max_vote = voteCount[k]
            predictedOutput = k
        
        allTestNeighbers.append(neighbors)
        allPredictedOutputs.append(predictedOutput)
        
    return allPredictedOutputs, allTestNeighbers
        
def performanceEvaluation(X_train, Y_train, X_test, Y_test, n_neighbors, obj, dataset_type):
    totalCount = 0
    correctCount = 0
    
    for testInput, testActualOutput in tqdm(zip(X_test, Y_test)):
        predictedOutput,_ = prediction(X_train, Y_train, [testInput], n_neighbors, obj)
        predict = predictedOutput[0]
        if predict == testActualOutput:
            correctCount += 1
        totalCount += 1
    msg = "For " + obj + "representation, " + "in dataset type: " + dataset_type + ", for k = " + str(n_neighbors) + ", Total Correct Count: " + str(correctCount) + " Total Wrong Count: " + str(totalCount-correctCount) + " Accuracy: " + str((correctCount*100)/(totalCount))
    print(msg)
    log(msg)
    return (correctCount*100)/(totalCount)


In [ ]:
max_accuracy = -999999
max_k = 1
for k in [1, 3, 5]:
  accuracy = performanceEvaluation(X_train_binary, Y_train_binary, X_valid_binary, Y_valid_binary, k, "binary", "validation")
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    max_k = k

msg = "For binary representation, maximum accuracy obtained with k = " + str(max_k) + ". \nRunning test set with k = " + str(max_k) + ":"
#performanceEvaluation(X_train_binary, Y_train_binary, X_test_binary, Y_test_binary, k, "binary", "test")

In [ ]:
del(X_train_binary)

In [ ]:
del(Y_train_binary)

In [ ]:
del(X_valid_binary)

In [ ]:
del(Y_valid_binary)

In [ ]:
max_accuracy = -999999
max_k = 1
for k in [1, 3, 5]:
  accuracy = performanceEvaluation(X_train_bow, Y_train_bow, X_valid_bow, Y_valid_bow, k, "bow", "validation")
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    max_k = k

msg = "For BOW representation, maximum accuracy obtained with k = " + str(max_k)
print(msg)
log(msg)
#performanceEvaluation(X_train_bow, Y_train_bow, X_test_bow, Y_test_bow, max_k, "bow", "test")

In [ ]:
max_accuracy = -999999
max_k = 1
for k in [1, 3, 5]:
  accuracy = performanceEvaluation(X_train_tfidf, Y_train_tfidf, X_valid_tfidf, Y_valid_tfidf, k, "tfidf", "validation")
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    max_k = k

msg = "For TFIDF representation, maximum accuracy obtained with k = " + str(max_k)
print(msg)
log(msg)
#performanceEvaluation(X_train_tfidf, Y_train_tfidf, X_test_tfidf, Y_test_tfidf, max_k, "tfidf", "test")

2200it [26:01,  1.41it/s]
0it [00:00, ?it/s]

For tfidfrepresentation, in dataset type: validation, for k = 1, Total Correct Count: 1783 Total Wrong Count: 417 Accuracy: 81.04545454545455


2200it [25:29,  1.44it/s]
0it [00:00, ?it/s]

For tfidfrepresentation, in dataset type: validation, for k = 3, Total Correct Count: 1832 Total Wrong Count: 368 Accuracy: 83.27272727272727


2200it [16:41,  2.20it/s]

For tfidfrepresentation, in dataset type: validation, for k = 5, Total Correct Count: 1847 Total Wrong Count: 353 Accuracy: 83.95454545454545
For TFIDF representation, maximum accuracy obtained with k = 5


In [ ]:
small_test_sets = {}
for i in range(1, 51):
  small_test_sets[i] = []

line_count = 0
test_set_count = 1
for line in tfidf_test_data:
  line_count += 1
  small_test_sets[test_set_count].append(line)
  if line_count % 10 == 0:
    test_set_count += 1
  if line_count % 500 == 0:
    test_set_count = 1 

In [ ]:
accuracy_list = []
for i in range(1, 51):
  test_set = small_test_sets[i]
  X_test = []
  Y_test = []
  for data in test_set:
    X_test.append(data['tfidf_doc'])
    Y_test.append(data['topic'])
  X_test = np.array(X_test)
  Y_test = np.array(Y_test)
  accuracy = performanceEvaluation(X_train_tfidf, Y_train_tfidf, X_test, Y_test, 5, "tfidf", "test")
  accuracy_list.append(accuracy)

knn_acc = {}
knn_acc['knn'] = accuracy_list

with open("knn_acc.json", "w") as f:
  json.dump(knn_acc, f)



0it [00:00, ?it/s]

1it [00:00,  2.08it/s]

2it [00:00,  2.11it/s]

3it [00:01,  2.15it/s]

4it [00:01,  2.18it/s]

5it [00:02,  2.13it/s]

6it [00:02,  2.16it/s]

7it [00:03,  2.17it/s]

8it [00:03,  2.19it/s]

9it [00:04,  2.19it/s]

10it [00:04,  2.21it/s]

11it [00:05,  2.18it/s]

12it [00:05,  2.20it/s]

13it [00:05,  2.20it/s]

14it [00:06,  2.21it/s]

15it [00:06,  2.22it/s]

16it [00:07,  2.21it/s]

17it [00:07,  2.22it/s]

18it [00:08,  2.22it/s]

19it [00:08,  2.21it/s]

20it [00:09,  2.20it/s]

21it [00:09,  2.22it/s]

22it [00:09,  2.22it/s]

23it [00:10,  2.22it/s]

24it [00:10,  2.22it/s]

25it [00:11,  2.22it/s]

26it [00:11,  2.23it/s]

27it [00:12,  2.19it/s]

28it [00:12,  2.18it/s]

29it [00:13,  2.16it/s]

30it [00:13,  2.19it/s]

31it [00:14,  2.16it/s]

32it [00:14,  2.19it/s]

33it [00:15,  2.19it/s]

34it [00:15,  2.19it/s]

35it [00:15,  2.19it/s]

36it [00:16,  2.20it/s]

37it [00:16,  2.21it/s]

38it [00:17,  2.21it/s]

39it [00:17,  2.23it/s]

40it [00:18,

For tfidfrepresentation, in dataset type: test, for k = 5, Total Correct Count: 81 Total Wrong Count: 29 Accuracy: 73.63636363636364




0it [00:00, ?it/s]

1it [00:00,  2.13it/s]

2it [00:00,  2.16it/s]

3it [00:01,  2.17it/s]

4it [00:01,  2.13it/s]

5it [00:02,  2.11it/s]

6it [00:02,  2.14it/s]

7it [00:03,  2.15it/s]

8it [00:03,  2.17it/s]

9it [00:04,  2.19it/s]

10it [00:04,  2.19it/s]

11it [00:05,  2.17it/s]

12it [00:05,  2.18it/s]

13it [00:05,  2.20it/s]

14it [00:06,  2.18it/s]

15it [00:06,  2.20it/s]

16it [00:07,  2.16it/s]

17it [00:07,  2.19it/s]

18it [00:08,  2.20it/s]

19it [00:08,  2.20it/s]

20it [00:09,  2.21it/s]

21it [00:09,  2.21it/s]

22it [00:10,  2.19it/s]

23it [00:10,  2.20it/s]

24it [00:11,  2.13it/s]

25it [00:11,  2.13it/s]

26it [00:11,  2.16it/s]

27it [00:12,  2.17it/s]

28it [00:12,  2.19it/s]

29it [00:13,  2.20it/s]

30it [00:13,  2.18it/s]

31it [00:14,  2.19it/s]

32it [00:14,  2.18it/s]

33it [00:15,  2.19it/s]

34it [00:15,  2.16it/s]

35it [00:16,  2.17it/s]

36it [00:16,  2.18it/s]

37it [00:16,  2.20it/s]

38it [00:17,  2.20it/s]

39it [00:17,  2.20it/s]

40it [00:18,

For tfidfrepresentation, in dataset type: test, for k = 5, Total Correct Count: 81 Total Wrong Count: 29 Accuracy: 73.63636363636364




0it [00:00, ?it/s]

1it [00:00,  1.98it/s]

2it [00:00,  2.04it/s]

3it [00:01,  2.08it/s]

4it [00:01,  2.11it/s]

5it [00:02,  2.15it/s]

6it [00:02,  2.15it/s]

7it [00:03,  2.15it/s]

8it [00:03,  2.14it/s]

9it [00:04,  2.17it/s]

10it [00:04,  2.19it/s]

11it [00:05,  2.19it/s]

12it [00:05,  2.14it/s]

13it [00:06,  2.17it/s]

14it [00:06,  2.17it/s]

15it [00:06,  2.15it/s]

16it [00:07,  2.15it/s]

17it [00:07,  2.16it/s]

18it [00:08,  2.19it/s]

19it [00:08,  2.19it/s]

20it [00:09,  2.18it/s]

21it [00:09,  2.19it/s]

22it [00:10,  2.21it/s]

23it [00:10,  2.21it/s]

24it [00:11,  2.23it/s]

25it [00:11,  2.25it/s]

26it [00:11,  2.24it/s]

27it [00:12,  2.25it/s]

28it [00:12,  2.23it/s]

29it [00:13,  2.20it/s]

30it [00:13,  2.20it/s]

31it [00:14,  2.21it/s]

32it [00:14,  2.21it/s]

33it [00:15,  2.23it/s]

34it [00:15,  2.15it/s]

35it [00:16,  2.07it/s]

36it [00:16,  2.06it/s]

37it [00:17,  2.08it/s]

38it [00:17,  2.12it/s]

39it [00:17,  2.14it/s]

40it [00:18,

...

For tfidfrepresentation, in dataset type: test, for k = 5, Total Correct Count: 73 Total Wrong Count: 37 Accuracy: 66.36363636363636




0it [00:00, ?it/s]

1it [00:00,  2.16it/s]

2it [00:00,  2.15it/s]

3it [00:01,  2.15it/s]

4it [00:01,  2.18it/s]

5it [00:02,  2.18it/s]

6it [00:02,  2.20it/s]

7it [00:03,  2.19it/s]

8it [00:03,  2.20it/s]

9it [00:04,  2.16it/s]

10it [00:04,  2.19it/s]

11it [00:05,  2.19it/s]

12it [00:05,  2.20it/s]

13it [00:05,  2.20it/s]

14it [00:06,  2.19it/s]

15it [00:06,  2.21it/s]

16it [00:07,  2.18it/s]

17it [00:07,  2.19it/s]

18it [00:08,  2.17it/s]

19it [00:08,  2.12it/s]

20it [00:09,  2.12it/s]

21it [00:09,  2.16it/s]

22it [00:10,  2.16it/s]

23it [00:10,  2.18it/s]

24it [00:11,  2.16it/s]

25it [00:11,  2.16it/s]

26it [00:11,  2.18it/s]

27it [00:12,  2.18it/s]

28it [00:12,  2.20it/s]

29it [00:13,  2.20it/s]

30it [00:13,  2.20it/s]

31it [00:14,  2.19it/s]

32it [00:14,  2.18it/s]

33it [00:15,  2.19it/s]

34it [00:15,  2.19it/s]

35it [00:16,  2.19it/s]

36it [00:16,  2.19it/s]

37it [00:16,  2.19it/s]

38it [00:17,  2.19it/s]

39it [00:17,  2.19it/s]

40it [00:18,

For tfidfrepresentation, in dataset type: test, for k = 5, Total Correct Count: 60 Total Wrong Count: 50 Accuracy: 54.54545454545455




0it [00:00, ?it/s]

1it [00:00,  2.04it/s]

2it [00:00,  2.08it/s]

3it [00:01,  2.08it/s]

4it [00:01,  2.12it/s]

5it [00:02,  2.12it/s]

6it [00:02,  2.15it/s]

7it [00:03,  2.16it/s]

8it [00:03,  2.18it/s]

9it [00:04,  2.14it/s]

10it [00:04,  2.16it/s]

11it [00:05,  2.17it/s]

12it [00:05,  2.17it/s]

13it [00:06,  2.18it/s]

14it [00:06,  2.18it/s]

15it [00:06,  2.19it/s]

16it [00:07,  2.19it/s]

17it [00:07,  2.20it/s]

18it [00:08,  2.17it/s]

19it [00:08,  2.19it/s]

20it [00:09,  2.20it/s]

21it [00:09,  2.21it/s]

22it [00:10,  2.21it/s]

23it [00:10,  2.20it/s]

24it [00:11,  2.16it/s]

25it [00:11,  2.15it/s]

26it [00:12,  2.12it/s]

27it [00:12,  2.11it/s]

28it [00:12,  2.14it/s]

29it [00:13,  2.14it/s]

30it [00:13,  2.17it/s]

31it [00:14,  2.14it/s]

32it [00:14,  2.18it/s]

33it [00:15,  2.19it/s]

34it [00:15,  2.19it/s]

35it [00:16,  2.20it/s]

36it [00:16,  2.19it/s]

37it [00:17,  2.20it/s]

38it [00:17,  2.20it/s]

39it [00:17,  2.21it/s]

40it [00:18,

For tfidfrepresentation, in dataset type: test, for k = 5, Total Correct Count: 73 Total Wrong Count: 37 Accuracy: 66.36363636363636


In [ ]:
import shutil
shutil.copy2("/content/knn_acc.json", "/content/drive/My Drive/ML Offline")

'/content/drive/My Drive/ML Offline/knn_acc.json'

In [ ]:
for i in range(len(accuracy_list)):
  print(str(i+1) + ": " + str(accuracy_list[i]))

1: 73.63636363636364
2: 73.63636363636364
3: 72.72727272727273
4: 61.81818181818182
5: 72.72727272727273
6: 72.72727272727273
7: 73.63636363636364
8: 71.81818181818181
9: 66.36363636363636
10: 63.63636363636363
11: 59.09090909090909
12: 68.18181818181819
13: 55.45454545454545
14: 62.72727272727273
15: 60.0
16: 66.36363636363636
17: 72.72727272727273
18: 66.36363636363636
19: 68.18181818181819
20: 60.0
21: 55.45454545454545
22: 70.0
23: 68.18181818181819
24: 65.45454545454545
25: 67.27272727272727
26: 66.36363636363636
27: 74.54545454545455
28: 60.0
29: 54.54545454545455
30: 61.81818181818182
31: 62.72727272727273
32: 62.72727272727273
33: 67.27272727272727
34: 64.54545454545455
35: 70.0
36: 66.36363636363636
37: 56.36363636363637
38: 59.09090909090909
39: 69.0909090909091
40: 64.54545454545455
41: 56.36363636363637
42: 65.45454545454545
43: 58.18181818181818
44: 70.9090909090909
45: 71.81818181818181
46: 60.90909090909091
47: 63.63636363636363
48: 66.36363636363636
49: 54.5454545454545